In [162]:
import os
import csv

In [163]:
# #random scribbling
# with open("propublica_full.csv", "r", encoding='utf-8') as file:
#     csvreader = csv.DictReader(file)
#     for row in csvreader:
#         #print(row["age_cat"])
        


In [165]:
def error(x,y,z):
    error = 0
    false_positive = 0
    false_positive_less25 = 0
    false_negative_less25 = 0
    ppv_less25 = 0
    false_positive_25_45 = 0
    false_negative_25_45 = 0
    ppv_25_45 = 0
    false_positive_great45 = 0
    false_negative_great45 = 0
    ppv_great45 = 0
    count = 6172
    with open("propublica_full.csv", "r", encoding='utf-8') as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            if row["age_cat"] == "Less than 25":
                if int(row["decile_score"]) >= x:      
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_less25 += 1
                    elif row["is_recid"] == "1":
                        ppv_less25 += 1
                elif int(row["decile_score"]) <= x:
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_less25 += 1
            if row["age_cat"] == "25 - 45":
                if int(row["decile_score"]) >= y:      
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_25_45 += 1
                    elif row["is_recid"] == "1":
                        ppv_25_45 += 1
                elif int(row["decile_score"]) <= y:
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_25_45 += 1
            if row["age_cat"] == "Greater than 45":
                if int(row["decile_score"]) >= z:      
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_great45 += 1
                    elif row["is_recid"] == "1":
                        ppv_great45 += 1
                elif int(row["decile_score"]) <= z:
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_great45 += 1
                        
    if false_positive_less25 == 0 and ppv_less25 == 0:
        ppv_less25 += 1
    if false_positive_25_45 == 0 and ppv_25_45 == 0:
        ppv_25_45 += 1
    if false_positive_great45 == 0 and ppv_great45 == 0:
        ppv_great45 += 1
    try:
        return {'totalerror' : 100*error/count, 
                'fp_low_mid': 100*abs((false_positive_less25/(false_positive_less25 + ppv_less25)
                                        - false_positive_25_45/(false_positive_25_45 + ppv_25_45))),
                'fp_mid_high': 100*abs((false_positive_25_45/(false_positive_25_45 + ppv_25_45)
                                        - false_positive_great45/(false_positive_great45 + ppv_great45))),
                'fp_low_high': 100*abs((false_positive_less25/(false_positive_less25 + ppv_less25)
                                        - false_positive_great45/(false_positive_great45 + ppv_great45)))}
        #return [100*error/count, 100*abs((false_positive_less25/(false_positive_less25 + ppv_less25) - false_positive_25_45/(false_positive_25_45 + ppv_25_45)))]
    except ZeroDivisionError:
        return [100*error/count,0]
    
    
                    


In [166]:
error(6,2,3)

{'totalerror': 39.43616331821128,
 'fp_low_mid': 13.747502849509146,
 'fp_mid_high': 1.3176545203802403,
 'fp_low_high': 12.429848329128907}

In [167]:
def errorFunc(x1, x2, x3, y):
 for i in range(11):
     for j in range(11):
         for k in range(11):
            returned = error(i,j,k)
            x1.append(returned["fp_low_mid"])
            x2.append(returned["fp_mid_high"])
            x3.append(returned["fp_low_high"])
            y.append(returned["totalerror"])

In [168]:
total_error = []
fp_low_mid = []
fp_mid_high = []
fp_low_high = []
errorFunc(fp_low_mid, fp_mid_high, fp_low_high, total_error)


In [174]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

# output to static HTML file
output_file("line.html")

data = {'x_values': total_error,
        'y_values': fp_low_high}


source = ColumnDataSource(data=data)

p = figure(plot_width=600, plot_height=600)

# add a circle renderer with a size, color, and alpha
p.circle(x='x_values', y='y_values', size=5, source=source, color="navy", alpha=0.5)

# show the results
show(p)